In [1]:
import sys
sys.path.insert(0, '../gambit')

import os
import numpy as np
import itertools

from gambit import create_diff_per_image, clean_data
from mask import create_segmentation_masks
from fits import read_fits, view_fits, write_fits

In [2]:
bands_in_order = ['g','r','i','z','y']

In [3]:
path_to_input = "/Users/cora-at-work/Desktop/github/spin-parity-catalog/original/galaxies/"
table_key = "table5"

def get_galaxy_list():
    return os.listdir(os.path.join(path_to_input,table_key))

def fits_path(name,band):
    return os.path.join(path_to_input,table_key,name,"{}_{}.fits".format(name,band))

In [4]:
def construct_diff_percentile(name,first_band,base_band):
    first_band_path = fits_path(name,first_band)
    base_band_path = fits_path(name,base_band)
    
    if not (os.path.exists(first_band_path) and os.path.exists(base_band_path)): 
        print("path does not exist guard")
        return (None,None)

    first_band_data = clean_data(read_fits(first_band_path)) #added clean to handle infs/ nans
    base_band_data = clean_data(read_fits(base_band_path)) #added clean to handle infs/ nans
    
    if first_band_data.shape != base_band_data.shape: 
        print("shape issue guard")
        return (None,None)
    
    (foreground,_) = create_segmentation_masks(base_band_data)
    
    if np.sum(foreground) <= 0.01*np.sum(np.shape(base_band_data)): 
        print("small forground guard")
        return (None,None)

    diff = create_diff_per_image(first_band_data,base_band_data,foreground)
    return (diff,foreground)

In [5]:
foreground_output_path = "/Users/cora-at-work/Desktop/research/winter_2023/foreground"
diff_per_output_path = "/Users/cora-at-work/Desktop/research/winter_2023/per_diff"

def get_output_path(name,first,base):
    return os.path.join(diff_per_output_path,table_key,"{}_diff_per_{}-{}.fits".format(name,first,base))

def get_foreground_path(name,base):
    return os.path.join(foreground_output_path,table_key,"{}_foreground_{}.fits".format(name,base))

In [6]:
i = 1
for name in get_galaxy_list():
    print(i,name)
    foreground_dict = {}
    for (first_band,base_band) in itertools.combinations(bands_in_order, 2):
        (diff,foreground) = construct_diff_percentile(name,first_band,base_band)
    
        if not isinstance(diff,np.ndarray):
            continue
        
        #write_path = get_output_path(name,first_band,base_band) #write each diff_per that exists
        #write_fits(write_path,diff_per)
        if base_band in foreground_dict or not isinstance(foreground,np.ndarray):
            continue
        foreground_dict[base_band] = foreground
            
    for base in foreground_dict: #write only one foreground mask per each choice of base
        foreground_path = get_foreground_path(name,base)
        write_fits(foreground_path,foreground_dict[base].astype('int')) #cast to int to save with write_fits
    i += 1
    #break

1 NGC3079
2 NGC3626
3 UGC8267
4 UGC8778
5 NGC3169
6 NGC4772
7 MCG-02-03-015
8 IC944
9 IC540
10 UGC6036
11 MCG-02-02-040
12 NGC6314
13 NGC681
14 IC2247
15 UGC10205
16 NGC4605
17 NGC1542
18 UGC5111
19 UGC5498
small forground guard
small forground guard
small forground guard
small forground guard
small forground guard
small forground guard
small forground guard
20 NGC4517
21 UGC9665
22 UGC3107
23 NGC3067
24 NGC3495
25 UGC10297
